In [1]:
# Load the STEP and extract 'solid' as before
from OCC.Extend.DataExchange import read_step_file
shape = read_step_file("0444-1.step")
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_SOLID
exp = TopExp_Explorer(shape, TopAbs_SOLID)
solid = exp.Current()

In [2]:
import numpy as np
import pandas as pd
from IPython.display import display

from OCC.Core.GProp          import    GProp_GProps
from OCC.Core.BRepGProp      import brepgprop
from OCC.Core.TopExp         import TopExp_Explorer
from OCC.Core.TopAbs         import TopAbs_FACE
from OCC.Core.BRepAdaptor    import BRepAdaptor_Surface
from OCC.Core.GeomAbs        import GeomAbs_Cylinder
from OCC.Core.TopoDS         import topods
from OCC.Core.gp             import gp_Pnt, gp_Dir, gp_Vec
from OCC.Core.BRepPrimAPI    import BRepPrimAPI_MakeSphere
from OCC.Core.Quantity       import Quantity_Color, Quantity_TOC_RGB
from OCC.Extend.ShapeFactory import get_oriented_boundingbox
from OCC.Display.SimpleGui   import init_display

def build_hole_table_and_view(solid):
    # --- 1) Get OBB results (3 or 4 items) ---
    obb_ret = get_oriented_boundingbox(solid)
    if len(obb_ret) == 4:
        bary, half_extents, obb_solid, axes = obb_ret
    else:
        bary, half_extents, obb_solid = obb_ret
        # fallback: compute axes via inertia
        props = GProp_GProps(); brepgprop.VolumeProperties(solid, props)
        im = props.MatrixOfInertia()
        T  = np.array([[im.Value(i,j) for j in (1,2,3)] for i in (1,2,3)])
        ev, evec = np.linalg.eigh(T)
        order = np.argsort(ev)
        axes = [gp_Dir(*evec[:,i]) for i in order]

    # --- 2) Convert bary (gp_Pnt or list) to a float array ---
    if hasattr(bary, "X"):
        centroid = np.array([bary.X(), bary.Y(), bary.Z()])
    else:
        centroid = np.array(bary, dtype=float)

    # --- 3) Convert axes to pure float vectors ---
    axes_dirs = []
    for d in axes:
        # d might be gp_Dir or some OCC vector
        xyz = d.XYZ() if hasattr(d, "XYZ") else d
        axes_dirs.append(np.array([xyz.X(), xyz.Y(), xyz.Z()], dtype=float))

    he = np.array([float(h) for h in half_extents])
    # identify long/flange/web indices
    long_idx = np.argmax(he)
    web_idx    = np.argmin(he)
    flange_idx = [0,1,2]
    flange_idx.remove(long_idx); flange_idx.remove(web_idx)
    flange_idx = flange_idx[0]

    # unpack half‐spans and axis vectors
    aL = he[long_idx]; aF = he[flange_idx]; aW = he[web_idx]
    L  = axes_dirs[ long_idx ]
    F  = axes_dirs[ flange_idx ]
    W  = axes_dirs[ web_idx    ]

    # --- 4) Compute NC1 origin at (H,U,O) corner ----
    origin_nc1 = centroid + aL*L - aW*W - aF*F

    # --- 5) Init the Qt viewer ---
    viewer, start_viewer, _, _ = init_display()
    grey = Quantity_Color(0.8,0.8,0.8,Quantity_TOC_RGB)
    viewer.DisplayShape(solid, color=grey, transparency=0.5, update=False)

    # --- 6) Find & classify holes ---
    tol_long, tol_web = 0.7, 0.7
    hole_data = []
    exp = TopExp_Explorer(solid, TopAbs_FACE)
    while exp.More():
        face = topods.Face(exp.Current())
        adp  = BRepAdaptor_Surface(face, True)
        if adp.GetType() == GeomAbs_Cylinder:
            cyl = adp.Cylinder()
            xyz = cyl.Axis().Direction().XYZ()
            g   = np.array([xyz.X(), xyz.Y(), xyz.Z()], dtype=float)
            ng  = np.linalg.norm(g)

            # skip holes drilled along length
            if abs(np.dot(g, L))/(ng*np.linalg.norm(L)) > tol_long:
                exp.Next(); continue

            # face centroid
            gp = GProp_GProps(); brepgprop.SurfaceProperties(face, gp)
            cm = gp.CentreOfMass()
            fc = np.array([cm.X(), cm.Y(), cm.Z()], dtype=float)

            # classify V vs O/U
            if abs(np.dot(g, W))/(ng*np.linalg.norm(W)) > tol_web:
                code, col = "V",(1,0,0)
            else:
                side = np.dot(fc - centroid, F)
                code, col = ("O",(0,1,0)) if side>0 else ("U",(0,0,1))

            hole_data.append((face, code, col, fc))
        exp.Next()

    # --- 7) Render holes & build the table ---
    rows = []
    for idx, (face, code, rgb, fc, cyl) in enumerate(hole_data, start=1):
        # draw face and sphere
        color_q = Quantity_Color(*rgb, Quantity_TOC_RGB)
        viewer.DisplayShape(face, color=color_q, update=False)
        viewer.DisplayShape(
            BRepPrimAPI_MakeSphere(gp_Pnt(*fc), 3).Shape(),
            color=color_q, update=False
        )
        # compute radial normal
        axis = cyl.Axis(); loc = axis.Location(); dirv = axis.Direction().XYZ()
        P0 = np.array([loc.X(), loc.Y(), loc.Z()])
        D  = np.array([dirv.X(), dirv.Y(), dirv.Z()], dtype=float)
        V0 = fc - P0
        radial = V0 - (np.dot(V0, D)/np.dot(D, D))*D
        normal = radial / np.linalg.norm(radial)
        # offset label in 3D
        offset_mm = 8.0
        pos = gp_Pnt(*(fc + offset_mm*normal))
        # correct DisplayMessage signature: (gp_Pnt, text, height, color, update)
        viewer.DisplayMessage(
            pos,
            str(idx),
            14,
            Quantity_Color(1,1,1,Quantity_TOC_RGB),
            False
        )
        # compute diameter and 2D coords
        diam = 2.0 * cyl.Radius()
        v2 = fc - origin_nc1
        if code == "V":
            x2d = float(np.dot(v2, L)); y2d = float(np.dot(v2, F))
        else:
            x2d = float(np.dot(v2, L)); y2d = float(np.dot(v2, W))
        rows.append({
            "Hole #": idx,
            "Code": code,
            "Diameter (mm)": round(diam,2),
            "X (mm)": round(x2d,2),
            "Y (mm)": round(y2d,2),
        })

    # origin marker
    yellow = Quantity_Color(1,1,0,Quantity_TOC_RGB)
    viewer.DisplayShape(
        BRepPrimAPI_MakeSphere(gp_Pnt(*origin_nc1), 5).Shape(),
        color=yellow, update=False
    )

    viewer.FitAll(); viewer.View_Iso(); start_viewer()
    return pd.DataFrame(rows)

# Usage:
df = build_hole_table_and_view(solid)
display(df)


pyqt5 backend - Qt version 5.15.2


ValueError: not enough values to unpack (expected 5, got 4)